# PARTE 0 | Importes Necessários

In [1]:
import kagglehub
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PowerTransformer

# PARTE 1 | Conjunto de Dados

In [3]:
# 1. Carregar Dados do Kaggle
def carregarDados(caminhoData: str) -> pd.DataFrame:
    """
    Carrega e unifica os arquivos CSV do dataset de HRV via 'uuid'.
    """

    dataframesBrutos = {}
    try:
        arquivosCsv = [f for f in os.listdir(caminhoData) if f.endswith('.csv')]
        for arquivo in arquivosCsv:
            nomeChave = arquivo.replace('.csv', '')
            dataframesBrutos[nomeChave] = pd.read_csv(os.path.join(caminhoData, arquivo))

        listaChaves = list(dataframesBrutos.keys())
        dfUnificado = dataframesBrutos[listaChaves[0]]
        for chave in listaChaves[1:]:
            dfUnificado = pd.merge(dfUnificado, dataframesBrutos[chave], on='uuid', how='inner')

        print(f"✅ Dataset carregado! E unificado:\n    {dfUnificado.shape}")
        return dfUnificado
    except Exception as e:
        print(f"❌ Erro no carregamento: {e}")
        return None


# 2. Seleção e Filtragem dos Dados
def selecionarPreditoresETarget(dfEntrada: pd.DataFrame):
    """
    Isola os preditores, remove classes intermediárias e equilibra as amostras
    para evitar viés na classificação binária.
    """

    # 1. Filtragem: Focar nos extremos (No Stress vs Time Pressure)
    # Removemos 'interruption' para limpar a fronteira de decisão
    dfLimpo = dfEntrada[dfEntrada['condition'] != 'interruption'].copy()

    # 2. Mapeamento Binário
    mapeamentoClasses = {'no stress': 0, 'time pressure': 1}
    y = dfLimpo['condition'].map(mapeamentoClasses).values


    # 3. Seleção de Features Estratégicas
    featuresFinais = [
                       'MEAN_RR', 'RMSSD', 'SDRR_REL_RR', 'RMSSD_REL_RR', # tempo
                       'VLF', 'LF', 'HF', 'LF_HF',                        # frequencia
                       'SD1', 'SD2', 'higuci'                             # nao linear
                      ]

    x = dfLimpo[featuresFinais].copy()

    print(f"➡️ Filtragem e mapeamento dos dados concluído.")

    return x, y


# 3. Divisão dos Conjuntos de Dados
def dividirDados(xDados, yAlvo, proporcaoTeste=0.2):
    """
    Apenas separa os dados em conjuntos de treino e teste de forma estratificada.
    """

    xTreino, xTeste, yTreino, yTeste = train_test_split(
        xDados, yAlvo, test_size=proporcaoTeste, random_state=42, stratify=yAlvo
    )
    print(f"➡️ Divisão dos conjuntos concluída (Teste: {proporcaoTeste*100}%).")
    return xTreino, xTeste, yTreino, yTeste


# 4. Transformação e normalização dos dados
def aplicarTransformacoes(xTreino, xTeste):
    """
    Aplica transformações para aproximar a distribuição normal e padroniza as escalas.
    Utiliza Yeo-Johnson para corrigir assimetria e StandardScaler para Z-score.
    """

    # 1. Transformação de Potência (Para aproximar da normalidade multivariada)
    # O método Yeo-Johnson é ideal para corrigir a assimetria (skewness) das features de HRV!!
    powerTrans = PowerTransformer(method='yeo-johnson')

    # 2. Padronização (Média 0, Variância 1)
    scaler = StandardScaler()

    # Aplicamos o pipeline de transformação
    # Nota: fit() apenas no treino para evitar vazamento de informação!
    xTreinoTrans = powerTrans.fit_transform(xTreino)
    xTreinoFinal = scaler.fit_transform(xTreinoTrans)

    xTesteTrans = powerTrans.transform(xTeste)
    xTesteFinal = scaler.transform(xTesteTrans)

    print("➡️ Transformação Yeo-Johnson e Padronização Z-score aplicadas.")
    return xTreinoFinal, xTesteFinal

In [4]:
# >> Pipeline de Execução

# 1. Download e definição de caminhos
path = kagglehub.dataset_download("vinayakshanawad/heart-rate-prediction-to-monitor-stress-level")
caminhoDados = os.path.join(path, 'Train Data', 'Train Data Zip')

Using Colab cache for faster access to the 'heart-rate-prediction-to-monitor-stress-level' dataset.


In [5]:
# 2. Ingestão e Unificação
dfHrv = carregarDados(caminhoDados)

if dfHrv is not None:
    # 3. Seleção de Features e Mapeamento do Target (Binário)
    # Aqui a função retorna os dados brutos ainda, apenas selecionados
    xBruto, yAlvo = selecionarPreditoresETarget(dfHrv)

    # 4. Divisão do Conjunto de Dados (Estratificada)
    # Fundamental dividir ANTES de qualquer transformação para evitar Data Leakage!!!
    xTreinoBruto, xTesteBruto, yTreino, yTeste = dividirDados(xBruto, yAlvo, proporcaoTeste=0.2)

    # 5. Transformação (Normalidade via Yeo-Johnson + Padronização Z-score)
    # Agora aplicamos a matemática para atender às premissas do LDA
    xTreino, xTeste = aplicarTransformacoes(xTreinoBruto, xTesteBruto)

    print("\n✅ Pipeline concluído com sucesso!")

✅ Dataset carregado! E unificado:
    (369289, 37)
➡️ Filtragem e mapeamento dos dados concluído.
➡️ Divisão dos conjuntos concluída (Teste: 20.0%).
➡️ Transformação Yeo-Johnson e Padronização Z-score aplicadas.

✅ Pipeline concluído com sucesso!
